Traditional value investment strategy

In [ ]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter
import math
from scipy import stats 

In [ ]:
stocks = pd.read_csv('sp500.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

In [ ]:
pe_ratio = data['peRatio']
pe_ratio

Build out the DataFrame using Batch API Call

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'Price-toEarnings Ratio', 'Number of Shares to Buy']

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol,
                                                    data[symbol]['quote']['latestPrice'],
                                                    data[symbol]['quote']['peRatio'],
                                                    'N/A'
                                                    ],
                                                    index = my_columns),
                                        ignore_index = True)
final_dataframe

Remove glamour stocks

In [ ]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-toEarnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)